<img src="images/dask_horizontal.svg" align="right" width="30%">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Hannah", "transactions": [{"transaction-id": 1638, "amount": 2761}, {"transaction-id": 2040, "amount": 2812}, {"transaction-id": 3293, "amount": 2757}, {"transaction-id": 4295, "amount": 2945}, {"transaction-id": 4732, "amount": 3060}, {"transaction-id": 7018, "amount": 3003}, {"transaction-id": 7134, "amount": 2923}]}\n',)

In [5]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text('s3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv',
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [6]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [7]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entires for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [8]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Hannah", "transactions": [{"transaction-id": 1638, "amount": 2761}, {"transaction-id": 2040, "amount": 2812}, {"transaction-id": 3293, "amount": 2757}, {"transaction-id": 4295, "amount": 2945}, {"transaction-id": 4732, "amount": 3060}, {"transaction-id": 7018, "amount": 3003}, {"transaction-id": 7134, "amount": 2923}]}\n',
 '{"id": 1, "name": "Victor", "transactions": [{"transaction-id": 1318, "amount": 133}, {"transaction-id": 6993, "amount": 165}]}\n',
 '{"id": 2, "name": "Ursula", "transactions": [{"transaction-id": 162, "amount": 1942}, {"transaction-id": 956, "amount": 1911}, {"transaction-id": 1136, "amount": 1739}, {"transaction-id": 1830, "amount": 2401}, {"transaction-id": 2214, "amount": 2088}, {"transaction-id": 3113, "amount": 2320}, {"transaction-id": 3581, "amount": 2068}, {"transaction-id": 4357, "amount": 1714}, {"transaction-id": 5710, "amount": 1914}, {"transaction-id": 9771, "amount": 2220}, {"transaction-id": 9991, "amount": 1934}]}\n')

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [9]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Hannah',
  'transactions': [{'transaction-id': 1638, 'amount': 2761},
   {'transaction-id': 2040, 'amount': 2812},
   {'transaction-id': 3293, 'amount': 2757},
   {'transaction-id': 4295, 'amount': 2945},
   {'transaction-id': 4732, 'amount': 3060},
   {'transaction-id': 7018, 'amount': 3003},
   {'transaction-id': 7134, 'amount': 2923}]},
 {'id': 1,
  'name': 'Victor',
  'transactions': [{'transaction-id': 1318, 'amount': 133},
   {'transaction-id': 6993, 'amount': 165}]},
 {'id': 2,
  'name': 'Ursula',
  'transactions': [{'transaction-id': 162, 'amount': 1942},
   {'transaction-id': 956, 'amount': 1911},
   {'transaction-id': 1136, 'amount': 1739},
   {'transaction-id': 1830, 'amount': 2401},
   {'transaction-id': 2214, 'amount': 2088},
   {'transaction-id': 3113, 'amount': 2320},
   {'transaction-id': 3581, 'amount': 2068},
   {'transaction-id': 4357, 'amount': 1714},
   {'transaction-id': 5710, 'amount': 1914},
   {'transaction-id': 9771, 'amount': 2220},
   {

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [10]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 3,
  'name': 'Alice',
  'transactions': [{'transaction-id': 249, 'amount': 1193},
   {'transaction-id': 289, 'amount': 1115},
   {'transaction-id': 380, 'amount': 1109},
   {'transaction-id': 792, 'amount': 1125},
   {'transaction-id': 1084, 'amount': 1155},
   {'transaction-id': 1217, 'amount': 1155},
   {'transaction-id': 1315, 'amount': 1161},
   {'transaction-id': 1500, 'amount': 1093},
   {'transaction-id': 2065, 'amount': 1072},
   {'transaction-id': 2296, 'amount': 1066},
   {'transaction-id': 2360, 'amount': 1113},
   {'transaction-id': 2652, 'amount': 1117},
   {'transaction-id': 3498, 'amount': 1073},
   {'transaction-id': 4242, 'amount': 1098},
   {'transaction-id': 4356, 'amount': 1089},
   {'transaction-id': 5060, 'amount': 1147},
   {'transaction-id': 5628, 'amount': 1128},
   {'transaction-id': 5651, 'amount': 1123},
   {'transaction-id': 6474, 'amount': 1116},
   {'transaction-id': 6801, 'amount': 1149},
   {'transaction-id': 6888, 'amount': 1054},
   {'transact

In [11]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 33},
 {'name': 'Alice', 'count': 41},
 {'name': 'Alice', 'count': 66},
 {'name': 'Alice', 'count': 21},
 {'name': 'Alice', 'count': 20})

In [12]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(33, 41, 66, 21, 20)

In [13]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

34.55950540958269

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [14]:
js.filter(lambda record: record['name'] == 'Alice').pluck('transactions').take(3)

([{'transaction-id': 249, 'amount': 1193},
  {'transaction-id': 289, 'amount': 1115},
  {'transaction-id': 380, 'amount': 1109},
  {'transaction-id': 792, 'amount': 1125},
  {'transaction-id': 1084, 'amount': 1155},
  {'transaction-id': 1217, 'amount': 1155},
  {'transaction-id': 1315, 'amount': 1161},
  {'transaction-id': 1500, 'amount': 1093},
  {'transaction-id': 2065, 'amount': 1072},
  {'transaction-id': 2296, 'amount': 1066},
  {'transaction-id': 2360, 'amount': 1113},
  {'transaction-id': 2652, 'amount': 1117},
  {'transaction-id': 3498, 'amount': 1073},
  {'transaction-id': 4242, 'amount': 1098},
  {'transaction-id': 4356, 'amount': 1089},
  {'transaction-id': 5060, 'amount': 1147},
  {'transaction-id': 5628, 'amount': 1128},
  {'transaction-id': 5651, 'amount': 1123},
  {'transaction-id': 6474, 'amount': 1116},
  {'transaction-id': 6801, 'amount': 1149},
  {'transaction-id': 6888, 'amount': 1054},
  {'transaction-id': 7330, 'amount': 1151},
  {'transaction-id': 7725, 'amount':

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 249, 'amount': 1193},
 {'transaction-id': 289, 'amount': 1115},
 {'transaction-id': 380, 'amount': 1109})

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(1193, 1115, 1109)

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

836.6685152057245

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [18]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [19]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [20]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [21]:
is_even = lambda x: x % 2
b.foldby(is_even, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [22]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 130), ('Alice', 166), ('Alice', 169), ('Alice', 182), ('Bob', 70), ('Bob', 91), ('Bob', 91), ('Bob', 98), ('Charlie', 58), ('Charlie', 77), ('Charlie', 81), ('Charlie', 83), ('Dan', 98), ('Dan', 129), ('Dan', 130), ('Dan', 139), ('Edith', 99), ('Edith', 130), ('Edith', 130), ('Edith', 139), ('Frank', 118), ('Frank', 148), ('Frank', 153), ('Frank', 167), ('George', 50), ('George', 65), ('George', 65), ('George', 70), ('Hannah', 50), ('Hannah', 65), ('Hannah', 65), ('Hannah', 70), ('Ingrid', 90), ('Ingrid', 117), ('Ingrid', 118), ('Ingrid', 126), ('Jerry', 112), ('Jerry', 145), ('Jerry', 147), ('Jerry', 160), ('Kevin', 80), ('Kevin', 104), ('Kevin', 104), ('Kevin', 112), ('Laura', 84), ('Laura', 111), ('Laura', 112), ('Laura', 120), ('Michael', 78), ('Michael', 310), ('Norbert', 125), ('Norbert', 158), ('Norbert', 161), ('Norbert', 173), ('Oliver', 90), ('Oliver', 117), ('Oliver', 117), ('Oliver', 126), ('Patricia', 129), ('Patricia', 167), ('Patricia', 348), ('Quinn', 77), ('

In [23]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot+1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 647), ('Bob', 350), ('Charlie', 299), ('Dan', 496), ('Edith', 498), ('Frank', 586), ('George', 250), ('Hannah', 250), ('Ingrid', 451), ('Jerry', 564), ('Kevin', 400), ('Laura', 427), ('Michael', 388), ('Norbert', 617), ('Oliver', 450), ('Patricia', 644), ('Quinn', 387), ('Ray', 686), ('Sarah', 350), ('Tim', 600), ('Ursula', 350), ('Victor', 381), ('Wendy', 534), ('Xavier', 383), ('Yvonne', 440), ('Zelda', 674)]
CPU times: user 194 ms, sys: 10.8 ms, total: 205 ms
Wall time: 657 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [24]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from for the bag point of view, they are frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [25]:
df1 = js.to_dataframe()
df1.head()

,id,name,transactions
0,0,Hannah,"[{'transaction-id': 1638, 'amount': 2761}, {'t..."
1,1,Victor,"[{'transaction-id': 1318, 'amount': 133}, {'tr..."
2,2,Ursula,"[{'transaction-id': 162, 'amount': 1942}, {'tr..."
3,3,Alice,"[{'transaction-id': 249, 'amount': 1193}, {'tr..."
4,4,Jerry,"[{'transaction-id': 28, 'amount': -1}, {'trans..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [26]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 271 ms, sys: 14.4 ms, total: 286 ms
Wall time: 1.86 s


name
Alice      647
Bob        350
Charlie    299
Dan        496
Edith      498
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [27]:
def denormalize(record):
    # returns a list for every nested item, each transaction of each person
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Hannah', 'amount': 2761, 'transaction-id': 1638},
 {'id': 0, 'name': 'Hannah', 'amount': 2812, 'transaction-id': 2040},
 {'id': 0, 'name': 'Hannah', 'amount': 2757, 'transaction-id': 3293})

In [28]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Hannah,2761,1638
1,0,Hannah,2812,2040
2,0,Hannah,2757,3293
3,0,Hannah,2945,4295
4,0,Hannah,3060,4732


In [29]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 264 ms, sys: 25 ms, total: 289 ms
Wall time: 1.63 s


name
Alice       22360
Bob          9827
Charlie      8440
Dan         15072
Edith       23656
Frank       22274
George      14846
Hannah       3147
Ingrid      18724
Jerry       25025
Kevin       16674
Laura       26578
Michael     22088
Norbert     20743
Oliver      21479
Patricia    28533
Quinn       12967
Ray         46976
Sarah       15772
Tim         21512
Ursula      11882
Victor       8320
Wendy       20823
Xavier      20496
Yvonne       7582
Zelda       34204
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [30]:
client.shutdown()